In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# Download data
currency_data = yf.download('USDINR=X', start='2020-01-01', end='2023-01-01')
currency_data = currency_data[['Close']].dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(currency_data.values)

def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i - time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [4]:
# Define objective function
def objective_function(hyperparams):
    lstm_units = int(hyperparams[0])
    learning_rate = hyperparams[1]
    batch_size = int(hyperparams[2])
    epochs = int(hyperparams[3])

    model = Sequential()
    model.add(tf.keras.layers.Input(shape=(X.shape[1], 1)))
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(LSTM(units=lstm_units, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)
    mse = model.evaluate(X, y, verbose=0)
    return mse

In [5]:
# BAS optimizer
class BASOptimizer:
    def __init__(self, objective_func, dim, bounds, max_iter=30):
        self.objective_func = objective_func
        self.dim = dim
        self.bounds = bounds
        self.max_iter = max_iter

    def optimize(self):
        x = np.random.uniform(self.bounds[:, 0], self.bounds[:, 1])
        d = 0.3
        delta = 0.1
        best_score = float('inf')
        best_params = x.copy()

        for t in range(self.max_iter):
            b = np.random.randn(self.dim)
            b = b / np.linalg.norm(b)

            xr = np.clip(x + d * b, self.bounds[:, 0], self.bounds[:, 1])
            xl = np.clip(x - d * b, self.bounds[:, 0], self.bounds[:, 1])

            fr = self.objective_func(xr)
            fl = self.objective_func(xl)

            x = np.clip(x + delta * b * np.sign(fr - fl), self.bounds[:, 0], self.bounds[:, 1])
            f = self.objective_func(x)

            if f < best_score:
                best_score = f
                best_params = x.copy()

            print(f"Iteration {t+1}/{self.max_iter} - Best MSE: {best_score:.8f}")
            d = 0.95 * d + 0.01
            delta *= 0.95

        return best_params, best_score

In [6]:
# Set hyperparameter search bounds
bounds = np.array([
    [10, 200],       # LSTM units
    [1e-4, 1e-2],    # Learning rate
    [16, 64],        # Batch size
    [5, 50]          # Epochs
])

In [7]:
# Run optimization
optimizer = BASOptimizer(objective_function, dim=4, bounds=bounds, max_iter=100)
best_params, best_score = optimizer.optimize()
print("Best Hyperparameters:", best_params)

Iteration 1/100 - Best MSE: 0.00077611
Iteration 2/100 - Best MSE: 0.00077611
Iteration 3/100 - Best MSE: 0.00077611
Iteration 4/100 - Best MSE: 0.00077611
Iteration 5/100 - Best MSE: 0.00077611
Iteration 6/100 - Best MSE: 0.00077611
Iteration 7/100 - Best MSE: 0.00063939
Iteration 8/100 - Best MSE: 0.00063939
Iteration 9/100 - Best MSE: 0.00063939
Iteration 10/100 - Best MSE: 0.00063939
Iteration 11/100 - Best MSE: 0.00063939
Iteration 12/100 - Best MSE: 0.00063939
Iteration 13/100 - Best MSE: 0.00062900
Iteration 14/100 - Best MSE: 0.00062900
Iteration 15/100 - Best MSE: 0.00062900
Iteration 16/100 - Best MSE: 0.00062900
Iteration 17/100 - Best MSE: 0.00062900
Iteration 18/100 - Best MSE: 0.00058435
Iteration 19/100 - Best MSE: 0.00058435
Iteration 20/100 - Best MSE: 0.00058435
Iteration 21/100 - Best MSE: 0.00058435
Iteration 22/100 - Best MSE: 0.00058435
Iteration 23/100 - Best MSE: 0.00058435
Iteration 24/100 - Best MSE: 0.00058435
Iteration 25/100 - Best MSE: 0.00058435
Iteration

In [8]:
# Train final model using optimized hyperparameters
final_units = int(best_params[0])
final_lr = float(best_params[1])
final_batch = int(best_params[2])
final_epochs = int(best_params[3])

In [9]:
final_model = Sequential()
final_model.add(tf.keras.layers.Input(shape=(X.shape[1], 1)))
final_model.add(LSTM(units=final_units, return_sequences=True))
final_model.add(LSTM(units=final_units, return_sequences=False))
final_model.add(Dropout(0.2))
final_model.add(Dense(units=1))

In [10]:
final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=final_lr), loss='mean_squared_error')
final_model.fit(X, y, epochs=final_epochs, batch_size=final_batch, verbose=1)

Epoch 1/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0840
Epoch 2/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0019
Epoch 3/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0025
Epoch 4/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0022
Epoch 5/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0016
Epoch 6/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0020
Epoch 7/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0017
Epoch 8/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0019
Epoch 9/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0018
Epoch 10/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0014
Epoch 11/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0018
Epoch 12/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0013
Epoch 13/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0014
Epoch 14/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0019
Epoch 15/36
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0020
Epoc

In [11]:
# Save the trained model
final_model.save("optimized_lstm_currency_model.keras")
print("optimized_lstm_currency_model.keras")

optimized_lstm_currency_model.keras
